<a href="https://colab.research.google.com/github/mostynw/shopify_intern_challenge/blob/main/Shopify_Technical_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Shopify Technical Challenge

In [2]:
#imports
import pandas as pd
import statistics as stats
import plotly.express as px

In [3]:
#g_sheets to pd DataFrame
sheet_url = 'https://docs.google.com/spreadsheets/d/16i38oonuX1y1g7C_UAmiK9GkY7cS-64DfiDMNiR41LM/edit#gid=0'
url_1 = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
df = pd.read_csv(url_1)
df.head(3)

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56


First thought when having a greater average than expected is that a few high valued orders are skewing the mean.

In [4]:
#boxplot of order amounts 
px.box(df, x='order_amount', title='Boxplot of Order Amounts')

There are clear outliers from the boxplot shown above.

In [5]:
#order amount histogram
px.histogram(df, x='order_amount', title='Order Amount Histogram')

In [6]:
df400 = df[df['order_amount']<1000]
px.histogram(df400, x='order_amount', title='Order Amount Histogram Zoomed')

In [7]:
len(df[df['order_amount']>5000])

63

The histogram shows that most orders have a value of under \$400 but 63 orders were over $5000 and 17 orders were between \$700-704.9k confirming that high-valued orders are skewing the AOV. Is this due to a high numbers of total items per order or because of a very expensive shoe? 

In [8]:
#total items histogram
px.histogram(df, x='total_items', title='Total Items Histogram')

99.66% of orders have an item total between 0-9. However 17 orders have item totals between 2000-2009! Lets check if there are any high-ticket items playing a part in skewing the AOV. 

In [9]:
#create a table of orders that are only one item
df1 = df[df['total_items']==1]
#check that every shop has had a sale of just 1 item. Every shop sells one item therefore each shop has one price it sells at
len(df1.groupby('shop_id')) #this value should be 100

100

In [10]:
#bar chart of shop prices
px.bar(df1, x='shop_id', y='order_amount', title='Shop Prices')

In [11]:
df1['order_amount'].max()

25725

shop_id = 78 may also be playing a part in skewing the AOV with its sale of sneakers at $25,725 each! 

In [12]:
#remove shop 78
df1_no78 = df1[df1['shop_id'] != 78]
#bar chart of shop prices
px.bar(df1_no78, x='shop_id', y='order_amount', title='Shop Prices without Shop 78')

A common method to calculating averages while minimising outliers influence is median. It takes the middle value. 

In [13]:
print('order_amount mean: ${:.2f}'.format(df['order_amount'].mean()))
print('order_amount median: ${:.2f}'.format(df['order_amount'].median()))

order_amount mean: $3145.13
order_amount median: $284.00


The AOV of \$284.00 seems much more appropriate than $3145.13

In [14]:
#remove shop 78
df_no78 = df[df['shop_id'] != 78]

print('order_amount mean: ${:.2f}'.format(df_no78['order_amount'].mean()))
print('order_amount median without shop 78: ${:.2f}'.format(df_no78['order_amount'].median()))

order_amount mean: $2717.37
order_amount median without shop 78: $284.00


Shop 78 has some influence on the mean but none on the median. Thus proving that the median would be a more reliable metric